In [1]:
import pandas as pd
from pathlib import Path

def excel_delta(Old_DS_Path, New_DS_Path, Index_Col, File_Type = 0, Old_DS_Sheet_Name = 0, New_DS_Sheet_Name = 0):
    
    """

    This function is used to calculate the delta between two excel files. Below is details on the fucntion's arguments:

        - Old_DS_Path: This is a required argument that specifies the path object specifying the directory of the old version of the file. Use the "from pathlib import Path" to import the "Path" library then create a Path object using "Old_DS_Path = Path('directory string')".
        - New_DS_Path: This is a required argument that specifies the path object specifying the directory of the new version of the file. Use the "from pathlib import Path" to import the "Path" library then create a Path object using "Old_DS_Path = Path('directory string')".
        - Index_Col: This is a required argument that specifies a list of the name(s) of the column(s) in the file that must be used as an index. It can either be an existing column in the file (the column values must be unique), or a combination of two or three columns.
          For example, if three columns must be used to generate the index, pass in a list of strings containing the column names as such ['alpha_2_code','alpha_3_code','numeric_code'].
        - File_Type: This is an optional argument that specifies if the file is an xlsx/xls or csv. By default the function expects an xls/xlsx file. If file type is csv, then pass in a value of 1.
        - Old_DS_Sheet_Name: This is an optional argument that specifies the name/numer of the sheet in the "Old" file that contains the data to be compared with the "New" file. By default, it is the first sheet in the file.
        - New_DS_Sheet_Name: This is an optional argument that specifies the name/numer of the sheet in the "New" file that contains the data to be compared with the "Old" file. By default, it is the first sheet in the file.

    """
    
    Change_Count = 0
    
    #Read files and replace NaNs with blank    
    if len(Index_Col) == 1:
        print("Reading files ...") 
        if File_Type == 0:
            Old_DS = pd.read_excel(Old_DS_Path, index_col=Index_Col[0], sheet_name=Old_DS_Sheet_Name).fillna('')
            New_DS = pd.read_excel(New_DS_Path, index_col=Index_Col[0], sheet_name=New_DS_Sheet_Name).fillna('')
        elif File_Type == 1:
            Old_DS = pd.read_csv(Old_DS_Path, index_col=Index_Col).fillna('')
            New_DS = pd.read_csv(New_DS_Path, index_col=Index_Col).fillna('')
        else:
            return(print("File type not compatible.\nOnly 'csv', 'xls' or 'xlsx' files accepted.\nChoose File_Type = 1 for 'csv' and 2 for 'xls' or 'xlsx'.\nExpects 'xls' or 'xlsx' by default."))

        print('\nIndex column: {}\n'.format(Index_Col[0]))
    
    elif len(Index_Col) == 2:
        
        print("Reading files ...") 
        if File_Type == 0:
            Old_DS = pd.read_excel(Old_DS_Path, sheet_name=Old_DS_Sheet_Name).fillna('')
            New_DS = pd.read_excel(New_DS_Path, sheet_name=New_DS_Sheet_Name).fillna('')
        elif File_Type == 1:
            Old_DS = pd.read_csv(Old_DS_Path).fillna('')
            New_DS = pd.read_csv(New_DS_Path).fillna('')
        else:
            return(print("File type not compatible.\nOnly 'csv', 'xls' or 'xlsx' files accepted.\nChoose File_Type = 1 for 'csv' and 2 for 'xls' or 'xlsx'.\nExpects 'xls' or 'xlsx' by default."))
        
        Old_DS['concat_key'] = ""
        New_DS['concat_key'] = ""
        
        for row in Old_DS.index:
            Old_DS.loc[row,'concat_key'] = ('{}{}').format(Old_DS.loc[row,Index_Col[0]],Old_DS.loc[row,Index_Col[1]])
            
        for row in New_DS.index:
            New_DS.loc[row,'concat_key'] = ('{}{}').format(New_DS.loc[row,Index_Col[0]],New_DS.loc[row,Index_Col[1]])
            
        New_DS['artificial_index'] = ""
        Old_DS['artificial_index'] = ""
        count = 0
        for rown in New_DS.index:
            for rowo in Old_DS.index:
                if New_DS.loc[rown,'concat_key'] == Old_DS.loc[rowo,'concat_key']:
                    New_DS.loc[rown,'artificial_index'] = count
                    Old_DS.loc[rowo,'artificial_index'] = count
                    count+=1
        
        #populate empty artificial_index values
        New_DS.artificial_index.replace('','blank',inplace=True)
        for rown in New_DS.index:
            if New_DS.loc[rown,'artificial_index'] == 'blank':
                New_DS.loc[rown,'artificial_index'] = count
                count+=1
        
        Old_DS.artificial_index.replace('','blank',inplace=True)
        for rowo in Old_DS.index:
            if Old_DS.loc[rowo,'artificial_index'] == 'blank':
                Old_DS.loc[rowo,'artificial_index'] = count
                count+=1
            
        New_DS.set_index('artificial_index',inplace=True)
        Old_DS.set_index('artificial_index',inplace=True)
        
    elif len(Index_Col) == 3:
        
        print("Reading files ...") 
        if File_Type == 0:
            Old_DS = pd.read_excel(Old_DS_Path, sheet_name=Old_DS_Sheet_Name).fillna('')
            New_DS = pd.read_excel(New_DS_Path, sheet_name=New_DS_Sheet_Name).fillna('')
        elif File_Type == 1:
            Old_DS = pd.read_csv(Old_DS_Path).fillna('')
            New_DS = pd.read_csv(New_DS_Path).fillna('')
        else:
            return(print("File type not compatible.\nOnly 'csv', 'xls' or 'xlsx' files accepted.\nChoose File_Type = 1 for 'csv' and 2 for 'xls' or 'xlsx'.\nExpects 'xls' or 'xlsx' by default."))
        
        Old_DS['concat_key'] = ""
        New_DS['concat_key'] = ""
        
        for row in Old_DS.index:
            Old_DS.loc[row,'concat_key'] = ('{}{}{}').format(Old_DS.loc[row,Index_Col[0]],Old_DS.loc[row,Index_Col[1]],Old_DS.loc[row,Index_Col[2]])
            
        for row in New_DS.index:
            New_DS.loc[row,'concat_key'] = ('{}{}{}').format(New_DS.loc[row,Index_Col[0]],New_DS.loc[row,Index_Col[1]],New_DS.loc[row,Index_Col[2]])
            
        New_DS['artificial_index'] = ""
        Old_DS['artificial_index'] = ""
        count = 0
        for rown in New_DS.index:
            for rowo in Old_DS.index:
                if New_DS.loc[rown,'concat_key'] == Old_DS.loc[rowo,'concat_key']:
                    New_DS.loc[rown,'artificial_index'] = count
                    Old_DS.loc[rowo,'artificial_index'] = count
                    count+=1
        
        #populate empty artificial_index values
        New_DS.artificial_index.replace('','blank',inplace=True)
        for rown in New_DS.index:
            if New_DS.loc[rown,'artificial_index'] == 'blank':
                New_DS.loc[rown,'artificial_index'] = count
                count+=1
        
        Old_DS.artificial_index.replace('','blank',inplace=True)
        for rowo in Old_DS.index:
            if Old_DS.loc[rowo,'artificial_index'] == 'blank':
                Old_DS.loc[rowo,'artificial_index'] = count
                count+=1
            
        New_DS.set_index('artificial_index',inplace=True)
        Old_DS.set_index('artificial_index',inplace=True)
        
    else:
        return(print('Index columns must be between 1 and 3 columns.'))

     #Calculate Delta
    print("Calculating delta ...")
    Delta = pd.DataFrame()
    Delta['Change Type'] = []
    Updated_DS = New_DS.copy()
    Deleted_Rows = []
    Added_Rows = []

    ##Determine common, added & deleted columns between the two files
    Old_DS_Cols = Old_DS.columns
    New_DS_Cols = New_DS.columns
    Common_Cols = list(set(Old_DS_Cols).intersection(New_DS_Cols))
    Added_Cols = list(set(New_DS_Cols).difference(Old_DS_Cols))
    Deleted_Cols = list(set(Old_DS_Cols).difference(New_DS_Cols))
    
    for row in New_DS.index:
        
        if (row in Old_DS.index) and (row in New_DS.index):
            #Check for delta in common columns
            for col in Common_Cols:
                Old_Value = Old_DS.loc[row,col]
                New_Value = New_DS.loc[row,col]
                if Old_Value==New_Value:
                    Updated_DS.loc[row,col] = New_DS.loc[row,col]
                else:
                    Delta.loc[row,col] = ('[{}] >>> [{}]').format(Old_Value,New_Value)
                    Delta.loc[row,'Change Type'] = 'Updated'
                    Updated_DS.loc[row,col] = ('{}').format(New_Value)
                    Change_Count += 1
        else:
            Added_Rows.append(row)
            Delta = Delta.append(New_DS.loc[row,:])
            Delta.loc[row,'Change Type'] = 'Added'

    for row in Old_DS.index:
        if row not in New_DS.index:
            Deleted_Rows.append(row)
            Delta = Delta.append(Old_DS.loc[row,:])
            Delta.loc[row,'Change Type'] = 'Deleted'

    Delta = Delta.fillna('')
    
    #Upcoming functionality to create updated file based on calculated delta
    #Updated_DS = Updated_DS.sort_index().fillna('')
    
    #Save Delta file
    print("Saving Delta file ...")
    DS_Path = Old_DS_Path.parent 
    Delta_DS_Path = str(DS_Path) + '\Delta.xlsx'
    Updated_DS_Path = str(DS_Path) + '\Modified_DS.xlsx'
    Delta.to_excel(Delta_DS_Path, index=True, header=True)
    Updated_DS.to_excel(Updated_DS_Path, index=True, header=True)
    #New_DS.to_excel(str(DS_Path) + '\New_DS',index=True, header=True)#Used for troubleshooting
    #Old_DS.to_excel(str(DS_Path) + '\Old_DS',index=True, header=True)#Used for troubleshooting
    
    #Print summary of changes
    print('\n-----[Change Summary]-----')
    print('\nAdded Rows: {}'.format(Added_Rows))
    print('Deleted Rows: {}'.format(Deleted_Rows))
    print('Added Columns: {}'.format(Added_Cols))
    print('Deleted Columns: {}'.format(Deleted_Cols))
    print('Change Count (Not including added/deleted rows/columns): {}'.format(Change_Count))  
    print('\nProcess Completed.')

In [9]:
Old_DS_Path = Path("C:\\Users\\Filename_v1.xls")
New_DS_Path = Path('C:\\Users\\Filename_v2.xls')
File_Type = 0
Old_DS_Sheet_Name = ''
New_DS_Sheet_Name = 0
excel_delta(Old_DS_Path, New_DS_Path,[0,1])

Reading files ...
Calculating delta ...
Saving Delta file ...

-----[Change Summary]-----

Added Rows: [17]
Deleted Rows: [18]
Added Columns: []
Deleted Columns: []
Change Count (Not including added/deleted rows/columns): 5

Process Completed.
